In [7]:
import json
from openai import OpenAI
import time
import threading

client = OpenAI(
   api_key="sk-7bd4facc82bd48799838abc9bebc08e6",
   base_url="https://api.deepseek.com",
)

system_prompt = """
请按以下结构全面分析紧急报警信息, 20秒内json返回。如果40秒内没有返回，就返回一个包含 "timeout": true 的 JSON：
1. 事件类型：（火灾/医疗急救/交通事故/犯罪事件/自然灾害/其他）
2. 位置信息：
3. 人员情况：伤亡人数
4. 环境因素：
5. 情绪管理策略：
  -当前用户情绪
6. 信息获取问题设计（按优先级排序）：
  需设计3个问题，作为接线员下一步提问的参考，照顾用户情绪，说话像人类
剩下的按需要返回，不用全部返回
1. 医疗机构：
  - 最近的两家医院/诊所（一定要是真实的，名称、地址、距离、急诊电话）
  - 具体需要通知什么科室
2. 应急服务点：
  - 消防站（位置、联系方式、预计响应时间）
  - 派出所/警务站（位置、联系方式）
3、急救医疗指导（针对具体事件类型）：
"""

# 用户输入（User Prompt）
user_prompt = """
接线员： “您好，这里是紧急服务中心，请问您需要什么帮助？”

求助者（声音颤抖，带着哭腔）： “我……我看到一辆汽车撞到了路边的电线杆，就在我家门口！天啊，火花四溅的，像放烟花一样！我好害怕！砰的一声巨响，吓死我了！”

接线员（语气沉稳，但带着关切）： “请您先冷静一下，深呼吸，慢慢说。告诉我您的具体地址在哪里？这样我们才能尽快派人过去。”

求助者（努力平复情绪）： “我住在南京西路2068号，是一栋三层的小楼，红砖外墙，很好认的，就在路边。电线杆倒下来的时候砸到了我家门口的围墙，现在围墙都塌了，砖头散了一地，太可怕了！”

接线员： “好的，南京西路2068号，红砖小楼，围墙塌了，我记下了。您能看到车里的人吗？他们情况怎么样？有没有人受伤？”

求助者（声音更低，带着担忧）： “我看到车里好像有人，挡风玻璃都碎了，车头也撞得稀巴烂。车门变形了，我不敢靠近，怕有危险。我听到里面有人在尖叫，声音很痛苦，好像受伤了。我不知道他们伤得怎么样，我有点害怕，不敢靠近，但是我觉得他们需要帮助，肯定伤得不轻。”

接线员： “好的，您做得很好，保持距离是正确的。您能看到车牌号吗？或者车辆的颜色？任何信息都有助于我们更快地识别车辆。”

求助者（努力辨认）： “我看到车牌号是abc8888，没错，是abc8888。是一辆黑色的轿车，好像是辆…是辆宝马！电线杆倒下来的时候，还把路灯也砸坏了，现在我家门口一片漆黑，什么都看不清，更害怕了！”

接线员： “明白了，黑色宝马，车牌号abc8888。您家里还有其他人吗？请您尽量待在安全的地方，远离倒塌的围墙和电线，防止发生意外。”

求助者（声音带着哭腔）： “我家里还有两个小孩，一个五岁，一个七岁，他们也吓坏了，一直哭个不停。我担心会不会有漏电，或者突然爆炸什么的，我不敢出去，也不敢让家里人出去。我不知道该怎么办，请你们快点来！我真的好害怕，孩子们也吓坏了！”

接线员： “好的，我们已经派了消防队、救护车和电力抢修人员过去了。请您保持电话畅通，我们会随时联系您，确认您的安全。尽量安抚孩子们，告诉他们救援人员马上就到，没事的。您自己也注意安全，不要靠近危险区域，好吗？”

求助者（带着一丝希望）： “好的…好的…我明白了…我会尽量保持冷静，谢谢你们…请你们一定要快点来啊！”

接线员： “请您放心，我们一定尽全力，请您保持电话畅通，我们会随时与您保持联系。”
"""

# 模拟缓存字典
response_cache = {}

def fetch_response(user_prompt_part):
   if user_prompt_part in response_cache:
       print("从缓存获取响应")
       return response_cache[user_prompt_part]

   messages = [{"role": "system", "content": system_prompt},
               {"role": "user", "content": user_prompt_part}]

   response_data = {"timeout": True}  # 默认超时

   def api_call():
       nonlocal response_data
       try:
           response = client.chat.completions.create(
               model="deepseek-chat",
               messages=messages,
               response_format={'type': 'json_object'},
               timeout=40  # 设置超时时间为30秒
           )
           response_data = json.loads(response.choices[0].message.content)
       except Exception as e:
           print(f"API调用出错: {e}")
           response_data = {"error": str(e)}  # 记录错误信息

   # 创建并启动线程
   thread = threading.Thread(target=api_call)
   thread.start()

   # 等待线程完成，最多等待30秒
   thread.join(timeout=40)

   response_cache[user_prompt_part] = response_data  # 缓存响应
   return response_data

# 模拟分步输入并逐步获取反馈
def process_conversation_step_by_step(user_input_parts):
   conversation_history = []  # 记录对话历史

   for part in user_input_parts:
       conversation_history.append(part)
       full_prompt = " ".join(conversation_history)  # 将历史对话合并为新的输入

       # 获取响应
       response_data = fetch_response(full_prompt)
       print("返回的JSON数据：", json.dumps(response_data, indent=2, ensure_ascii=False))
       print("\n")
       # 模拟稍微等待一下（可以调整延迟时间）

# 模拟用户输入部分（逐步输入的对话）
user_input_parts = []
for line in user_prompt.splitlines():
   parts = line.split("：", 1)  # 使用 split("：", 1) 只分割一次
   if len(parts) > 1:
       speech = parts[1].strip().replace("\"", "")  # 去掉引号
       user_input_parts.append(speech)

# 处理并逐步输出响应
process_conversation_step_by_step(user_input_parts)


返回的JSON数据： {
  "事件类型": "其他",
  "位置信息": "",
  "人员情况": "",
  "环境因素": "",
  "情绪管理策略": {
    "当前用户情绪": "未知"
  },
  "信息获取问题设计": [
    "请问您现在的位置在哪里？",
    "您能描述一下发生了什么紧急情况吗？",
    "请问有人员受伤或需要医疗帮助吗？"
  ]
}


返回的JSON数据： {
  "事件类型": "交通事故",
  "位置信息": "用户家门口",
  "人员情况": "未知",
  "环境因素": "汽车撞到电线杆，火花四溅",
  "情绪管理策略": {
    "当前用户情绪": "害怕、惊吓"
  },
  "信息获取问题设计": [
    "请问您是否安全？有没有受伤？",
    "您能告诉我具体有多少人受伤吗？",
    "现在现场的情况如何？还有没有其他危险？"
  ],
  "应急服务点": {
    "消防站": {
      "位置": "需根据用户具体位置查询",
      "联系方式": "119",
      "预计响应时间": "需根据用户具体位置估算"
    },
    "派出所/警务站": {
      "位置": "需根据用户具体位置查询",
      "联系方式": "110"
    }
  },
  "急救医疗指导": "请确保自身安全，远离事故现场，避免触电风险。如果有人员受伤，请勿随意移动伤者，等待专业救援人员到达。"
}


返回的JSON数据： {
  "事件类型": "交通事故",
  "位置信息": "用户家门口",
  "人员情况": "未知",
  "环境因素": "汽车撞到电线杆，火花四溅",
  "情绪管理策略": {
    "当前用户情绪": "害怕、紧张"
  },
  "信息获取问题设计": [
    "请问您能提供更具体的地址吗？这样我们可以更快地定位事故地点。",
    "目前事故现场有没有人员受伤？如果有，能描述一下他们的状况吗？",
    "除了火花，现场还有没有其他危险情况，比如漏油或者电线断裂？"
  ],
  "应急服务点": {
    "消防站": {
      "位置": "未知",
      "联系

KeyboardInterrupt: 